In [42]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping

In [2]:
os.chdir('D:\\ai\\dataset\\dogs-vs-cats')
if os.path.isdir('train/dog') is False:
    os.makedirs('train/dog')
    os.makedirs('train/cat')
    os.makedirs('valid/dog')
    os.makedirs('valid/cat')
    os.makedirs('test/dog')
    os.makedirs('test/cat')
for i in random.sample(glob.glob('cat*'),1000):
    shutil.move(i,'train/cat')
for i in random.sample(glob.glob('dog*'),1000):
    shutil.move(i,'train/dog')

for i in random.sample(glob.glob('cat*'),200):
    shutil.move(i,'valid/cat')
for i in random.sample(glob.glob('dog*'),200):
    shutil.move(i,'valid/dog')

for i in random.sample(glob.glob('cat*'),100):
    shutil.move(i,'test/cat')
for i in random.sample(glob.glob('dog*'),100):
    shutil.move(i,'test/dog')    

In [3]:
train_path='D:\\ai\\dataset\\dogs-vs-cats\\train'
valid_path='D:\\ai\\dataset\\dogs-vs-cats\\valid'
test_path ='D:\\ai\\dataset\\dogs-vs-cats\\test'

In [4]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(224,224), classes=['cat', 'dog'], batch_size=10)

valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=valid_path, target_size=(224,224), classes=['cat', 'dog'], batch_size=10)

test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(224,224), classes=['cat', 'dog'], batch_size=10, shuffle=False)

Found 8000 images belonging to 2 classes.
Found 1600 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [22]:
vgg16_model=keras.applications.vgg16.VGG16()

In [23]:
vgg16_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [34]:
vgg16_model.layers.pop()
vgg16_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [29]:
for layer in vgg16_model.layers:
    layer.trainable=False

In [35]:
#model=Sequential()
#model.add(vgg16_model)
#model.add(Dense(2,activation='softmax'))
#model.summary()
predictions = Dense(2, activation = "softmax")(vgg16_model.layers[-2].output)
model = Model(inputs=vgg16_model.input, outputs=predictions)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [37]:
model.compile(optimizer="RMSprop",loss="categorical_crossentropy",metrics ="accuracy")

In [44]:
model.fit( train_batches,
          epochs=10,
          validation_data=valid_batches,
          validation_steps=1,
          callbacks = [EarlyStopping(monitor = 'val_accuracy', patience = 20)]
         )

Epoch 1/10
 33/800 [>.............................] - ETA: 22:37 - loss: 0.1035 - accuracy: 0.9727

KeyboardInterrupt: 